In [ ]:
import pandas as pd
import scanpy as sc
import json
import requests

# Preprocessing

## Gene Names

In [ ]:
df_conversion = pd.read_csv("https://www.genenames.org/cgi-bin/download/custom?col=gd_app_sym&col=md_ensembl_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit", sep="\t").set_index("Approved symbol")
df_conversion.rename(columns={"Ensembl ID(supplied by Ensembl)":"ensg"}, inplace=True)
df_conversion.head(2)

## bioMART
https://www.ensembl.org/biomart/martview/cf9b27b6e78e9d6a1be079a4ea60f7fe

In [ ]:
df_mart = pd.read_csv("mart_export.txt").set_index("Gene stable ID")
df_mart.head(2)

# TCGA FPKM

In [ ]:
df_tcga = pd.read_csv("mainTable_fpkm.csv", sep=",", index_col=0)
df_tcga = df_tcga.transpose().drop_duplicates().transpose()
df_tcga.head(2)

In [ ]:
# isolate pc
df_tcga = df_tcga.join(df_mart, how="inner")
df_tcga = df_tcga[df_tcga["Gene type"]=="protein_coding"]
df_tcga = df_tcga.drop(["miRBase ID", "Gene type"],1)
cases = ["-".join(case.split("-")[:5]) for case in df_tcga.columns]
analytes = ["-".join(case.split("-")[:6]) for case in df_tcga.columns]
df_tcga.head(2)

In [ ]:
df_files = pd.read_csv("files.dat")
df_files = df_files[df_files["sample_submitter_id"].isin(df_tcga.columns)]
df_files = df_files.loc[df_files.index[~df_files["sample_submitter_id"].duplicated(keep="first")],:]
df_files = df_files.set_index(df_files.columns[0])
df_files.head()

## HVG

In [ ]:
adata = sc.AnnData(
    X=df_tcga.transpose().reindex(index=df_files["sample_submitter_id"]), 
    obs=df_files.set_index("sample_submitter_id"))

In [ ]:
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=3000)
sc.pl.highly_variable_genes(adata)

In [ ]:
hvg = adata.var[adata.var["highly_variable"]==True].index

## miRNA

In [ ]:
df_tcga_mirna = pd.read_csv("mainTable_mirna.csv", index_col=0)
df_tcga_mirna = df_tcga_mirna.transpose().drop_duplicates().transpose()
mirna_submitter_ids = ["-".join(case.split("-")[:4]) for case in df_tcga_mirna.columns]
df_tcga_mirna.head(2)

## Highly variable miRNA

In [ ]:
adata = sc.AnnData(
    X=df_tcga_mirna.transpose().reindex(index=df_files[df_files["sample_submitter_id_mirna"].isin(df_tcga_mirna.columns)]["sample_submitter_id_mirna"]), 
    obs=df_files[df_files["sample_submitter_id_mirna"].isin(df_tcga_mirna.columns)].set_index("sample_submitter_id_mirna"))

In [ ]:
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, n_top_genes=1000)
sc.pl.highly_variable_genes(adata)

In [ ]:
hvmiRNA = adata.var[adata.var["highly_variable"]==True].index.unique()

In [ ]:
df_files[["sample_submitter_id", "sample_submitter_id_mirna"]]

In [ ]:
df_tcga_mirna = df_tcga_mirna.reindex(columns=df_files["sample_submitter_id_mirna"])
df_tcga_mirna.columns = df_files.index
df_tcga = df_tcga.reindex(columns=df_files["sample_submitter_id"])
df_tcga.columns = df_files.index

In [ ]:
df_all = df_tcga.reindex(index=hvg, columns=df_files.index).append(df_tcga_mirna.reindex(index=hvmiRNA))
df_all

In [ ]:
df_files.to_csv("files.dat")

# hSBM

In [ ]:
import sys
sys.path.append("../hSBM_Topicmodel/")
from sbmtm import sbmtm

In [ ]:
hsbm = sbmtm()
hsbm.make_graph_from_BoW_df(df_tcga.reindex(index=hvg))

In [ ]:
hsbm.g

In [ ]:
hsbm.save_graph("graph_breast_hsbm.xml.gz")

## triSBM

In [ ]:
sys.path.append("../trisbm/")
from trisbm import trisbm

In [ ]:
trisbm = trisbm()
trisbm.make_graph(df_all, lambda gene: 1 if "ENSG" in gene else 2)

In [ ]:
trisbm.g

In [ ]:
trisbm.save_graph("graph_breast_trisbm.xml.gz")